In [1]:
using JLD2, FileIO
using Statistics, Core
#using Flux: onehotbatch, onecold, crossentropy, throttle
using Base.Iterators: repeated


In [2]:
f = jldopen("../data/ocean_convection_profiles.jld2","r+")

JLDFile C:\Users\janak\Dropbox (MIT)\6.S898\6S898-climate-parameterization\data\ocean_convection_profiles.jld2 (read/write)
 ├─📂 parameters
 │  ├─🔢 coriolis_parameter
 │  ├─🔢 density
 │  ├─🔢 specific_heat_capacity
 │  ├─🔢 viscosity
 │  ├─🔢 diffusivity
 │  ├─🔢 diffusivity_T
 │  ├─🔢 diffusivity_S
 │  ├─🔢 surface_cooling
 │  ├─🔢 temperature_gradient
 │  └─🔢 wind_stress_x
 ├─📂 boundary_conditions
 │  ├─📂 top
 │  │  ├─🔢 FT
 │  │  └─🔢 Fu
 │  └─📂 bottom
 │     └─🔢 dTdz
 ├─📂 grid
 │  ├─🔢 Nx
 │  ├─🔢 Ny
 │  ├─🔢 Nz
 │  ├─🔢 Hx
 │  ├─🔢 Hy
 │  ├─🔢 Hz
 │  ├─🔢 Tx
 │  ├─🔢 Ty
 │  ├─🔢 Tz
 │  ├─🔢 Lx
 │  ├─🔢 Ly
 │  ├─🔢 Lz
 │  ├─🔢 Δx
 │  ├─🔢 Δy
 │  ├─🔢 Δz
 │  ├─🔢 xC
 │  ├─🔢 yC
 │  ├─🔢 zC
 │  ├─🔢 xF
 │  ├─🔢 yF
 │  └─🔢 zF
 ├─📂 coriolis
 │  └─🔢 f
 ├─📂 buoyancy
 │  ├─🔢 gravitational_acceleration
 │  └─📂 equation_of_state
 │     ├─🔢 α
 │     └─🔢 β
 ├─📂 closure
 │  ├─🔢 C
 │  ├─🔢 Cb
 │  ├─🔢 ν
 │  └─📂 κ
 │     ├─🔢 T
 │     └─🔢 S
 └─📂 timeseries
    ├─📂 t
    │  ├─🔢 0
    │  ├─🔢 161
    │  ├─🔢 262
    │  ├─🔢 335
   

In [3]:
grid_res = length(f["timeseries/T/0"])

258

In [10]:
convert_res = 32
#@assert grid_res % convert_res == 0
#a = [1,2,3]
#a[1:-2]

0-element Array{Int64,1}

In [11]:
function convert_to_low_res(convert_res, data)
    if length(data) == 1
        return data
    else
        data = data[2:length(data)-1]
        @assert length(data) % convert_res == 0
        s = length(data)/convert_res
        temp_array = zeros(convert_res)
        for i in 1:convert_res
            t = data[Int((i-1)*s+1):Int(i*s)]
            temp_array[i] = Statistics.mean(t)
        end
        return temp_array
    end
end

convert_to_low_res (generic function with 1 method)

In [12]:
function change_res(convert_res, f)
    low_res_data = Dict()
    for k in keys(f["timeseries"])
        low_res_data[k] = Dict()
        for k_iter in keys(f["timeseries"][k])
            #println(k,k_iter)
            try
                low_res_data[k][k_iter] = convert_to_low_res(convert_res, f["timeseries"][k][k_iter])
            catch y
                println(k,k_iter)
                println(y)
                break
            end
        end
    end
    return low_res_data
end

change_res (generic function with 1 method)

In [13]:
low_res_data = change_res(convert_res, f)

Dict{Any,Any} with 18 entries:
  "nu"     => Dict{Any,Any}("517"=>[1.05e-6, 1.05e-6, 1.05e-6, 1.05e-6, 1.05e-6…
  "wT"     => Dict{Any,Any}("517"=>[-1.18129e-25, -1.31491e-24, -4.13174e-24, -…
  "T"      => Dict{Any,Any}("517"=>[19.0156, 19.0469, 19.0781, 19.1094, 19.1406…
  "vv"     => Dict{Any,Any}("517"=>[1.76522e-24, 1.99782e-24, 2.50094e-24, 3.38…
  "uv"     => Dict{Any,Any}("517"=>[-1.11951e-25, -1.27032e-25, -1.6278e-25, -2…
  "ww"     => Dict{Any,Any}("517"=>[3.75842e-25, 1.13463e-24, 2.75015e-24, 5.61…
  "t"      => Dict{Any,Any}("517"=>4209.58,"6420"=>49201.2,"7544"=>55205.2,"600…
  "vw"     => Dict{Any,Any}("517"=>[1.35146e-27, 6.57009e-27, 1.80087e-26, 4.35…
  "kappaS" => Dict{Any,Any}("517"=>[1.46e-7, 1.46e-7, 1.46e-7, 1.46e-7, 1.46e-7…
  "wS"     => Dict{Any,Any}("517"=>[-9.92791e-26, -1.23321e-24, -4.0223e-24, -9…
  "v"      => Dict{Any,Any}("517"=>[9.00213e-24, -1.55146e-23, 6.03207e-24, -1.…
  "kappaT" => Dict{Any,Any}("517"=>[1.46e-7, 1.46e-7, 1.46e-7, 1.46e-7, 1.46e-

In [17]:
jldopen("32xres_data_with_dTdt.jld2","w+") do file
    file["timeseries"] = low_res_data
end

Dict{Any,Any} with 18 entries:
  "nu"     => Dict{Any,Any}("517"=>[1.05e-6, 1.05e-6, 1.05e-6, 1.05e-6, 1.05e-6…
  "wT"     => Dict{Any,Any}("517"=>[-1.18129e-25, -1.31491e-24, -4.13174e-24, -…
  "T"      => Dict{Any,Any}("517"=>[19.0156, 19.0469, 19.0781, 19.1094, 19.1406…
  "vv"     => Dict{Any,Any}("517"=>[1.76522e-24, 1.99782e-24, 2.50094e-24, 3.38…
  "uv"     => Dict{Any,Any}("517"=>[-1.11951e-25, -1.27032e-25, -1.6278e-25, -2…
  "ww"     => Dict{Any,Any}("517"=>[3.75842e-25, 1.13463e-24, 2.75015e-24, 5.61…
  "t"      => Dict{Any,Any}("517"=>4209.58,"6420"=>49201.2,"7544"=>55205.2,"600…
  "vw"     => Dict{Any,Any}("517"=>[1.35146e-27, 6.57009e-27, 1.80087e-26, 4.35…
  "kappaS" => Dict{Any,Any}("517"=>[1.46e-7, 1.46e-7, 1.46e-7, 1.46e-7, 1.46e-7…
  "wS"     => Dict{Any,Any}("517"=>[-9.92791e-26, -1.23321e-24, -4.0223e-24, -9…
  "v"      => Dict{Any,Any}("517"=>[9.00213e-24, -1.55146e-23, 6.03207e-24, -1.…
  "kappaT" => Dict{Any,Any}("517"=>[1.46e-7, 1.46e-7, 1.46e-7, 1.46e-7, 1.46e-